In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('ProjectData.csv')
text_column = 'Name'
target_column = 'normalizedHours'

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df[text_column].tolist(), 
    df[target_column].tolist(), 
    test_size=0.2, 
    random_state=42
)

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

In [ ]:
from torch.utils.data import DataLoader, Dataset

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)


In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import root_mean_squared_error

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,  
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=100, 
    weight_decay=0.1,
    save_total_limit=2, 
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-6,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.squeeze() 
    rmse = root_mean_squared_error(labels, preds)
    return {"rmse": rmse}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
#trainer.train()
#Only uncomment if you are absolutely sure you don't need your GPU for the next hour